## The following section is for Colab Users.
### Just run the following code cells

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 44.4 MB/s 
     |████████████████████████████████| 154 kB 51.9 MB/s 


In [ ]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar -P /content/spark-3.0.2-bin-hadoop2.7/jars/
!cp /content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.2-bin-hadoop2.7/graphframes-0.8.2-spark3.0-s_2.12.zip

--2022-07-13 20:28:18--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 54.192.18.81, 54.192.18.30, 54.192.18.122, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|54.192.18.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247882 (242K) [binary/octet-stream]
Saving to: ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.04s   

2022-07-13 20:28:18 (6.20 MB/s) - ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’ saved [247882/247882]



In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [ ]:
import findspark
findspark.init()

In [ ]:
!export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"
!export PYSPARK_DRIVER_PYTHON=jupyter
!export PYSPARK_DRIVER_PYTHON_OPTS=notebook

In [ ]:
from pyspark.sql import SparkSession
from graphframes import *

spark = SparkSession.builder.master("local[*]").appName("GraphFrames").getOrCreate()

In [ ]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

**************************************************************************
**************************************************************************
**************************************************************************

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import split , explode
from pyspark.sql.types import *

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read departuredelays.csv in Edge DataFrame
### Read airport-codes-na.txt in Vertix DataFrame (the separator is Tab i.e sep = '\t' )

#### The US flight delays data set has five columns:
- The <b>date</b> column contains an integer like 02190925 . When converted, this maps to 02-19 09:25 am.
- The <b>delay</b> column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The <b>distance</b> column gives the distance in miles from the origin airport to the destination airport.
- The <b>origin</b> column contains the origin IATA airport code.
- The <b>destination</b> column contains the destination IATA airport code.

#### The airport-codes data set has four columns:
- The <b>IATA</b> column contains IATA airport code.
- The <b>City, State, and Country</b> columns contains information about the airport location. 

In [ ]:
vertx = spark.read.csv("/content/airport-codes-na.txt", header = True, inferSchema='true', sep='\t')

In [ ]:
vertx.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [ ]:
print(vertx.count())

526


### In the vertix DataFrame, drop any duplicated rows with the same  IATA code.

In [ ]:
vertx = vertx.dropDuplicates(['IATA'])
vertx.show()

+-------------------+-----+-------+----+
|               City|State|Country|IATA|
+-------------------+-----+-------+----+
|         Binghamton|   NY|    USA| BGM|
|            Lebanon|   NH|    USA| LEB|
|           Montreal|   PQ| Canada| YUL|
|         Dillingham|   AK|    USA| DLG|
|International Falls|   MN|    USA| INL|
|         Wolf Point|   MT|    USA| OLF|
|        New Orleans|   LA|    USA| MSY|
|            Toronto|   ON| Canada| YTO|
|            Spokane|   WA|    USA| GEG|
|              Havre|   MT|    USA| HVR|
|            Burbank|   CA|    USA| BUR|
|      Orange County|   CA|    USA| SNA|
|             Dryden|   ON| Canada| YHD|
|         Fort Dodge|   IA|    USA| FOD|
|          Green Bay|   WI|    USA| GRB|
|        Great Falls|   MT|    USA| GTF|
|              Homer|   AK|    USA| HOM|
|        Idaho Falls|   ID|    USA| IDA|
|      Sioux Lookout|   ON| Canada| YXL|
|       Grand Rapids|   MI|    USA| GRR|
+-------------------+-----+-------+----+
only showing top

In [ ]:
print(vertx.count())

524


### In the edges DataFrame:
- Rename the <b>date</b> columns to become <b>tripid</b>.
- Rename the <b>origin</b> columns to become <b>src</b>.
- Rename the <b>destination</b> columns to become <b>dst</b>.

In [ ]:
edges = spark.read.csv("/content/departuredelays.csv", header= True, inferSchema= True)
edges.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [ ]:
edges = edges.withColumnRenamed("date","tripid")\
.withColumnRenamed("origin","src")\
.withColumnRenamed("destination","dst")
edges.printSchema()

root
 |-- tripid: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)



### In the Vertix DataFrame:
- Rename the <b>IATA</b> columns to become <b>id</b>.

In [ ]:
vertx = vertx.withColumnRenamed("IATA","id")
vertx.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- id: string (nullable = true)



### Create GraphFrame from Vertix and Edges DataFrames

In [ ]:
gf1 = GraphFrame(vertx, edges)

In [ ]:
gf1.vertices.show()

+-------------------+-----+-------+---+
|               City|State|Country| id|
+-------------------+-----+-------+---+
|         Binghamton|   NY|    USA|BGM|
|            Lebanon|   NH|    USA|LEB|
|           Montreal|   PQ| Canada|YUL|
|         Dillingham|   AK|    USA|DLG|
|International Falls|   MN|    USA|INL|
|         Wolf Point|   MT|    USA|OLF|
|        New Orleans|   LA|    USA|MSY|
|            Toronto|   ON| Canada|YTO|
|            Spokane|   WA|    USA|GEG|
|              Havre|   MT|    USA|HVR|
|            Burbank|   CA|    USA|BUR|
|      Orange County|   CA|    USA|SNA|
|             Dryden|   ON| Canada|YHD|
|         Fort Dodge|   IA|    USA|FOD|
|          Green Bay|   WI|    USA|GRB|
|        Great Falls|   MT|    USA|GTF|
|              Homer|   AK|    USA|HOM|
|        Idaho Falls|   ID|    USA|IDA|
|      Sioux Lookout|   ON| Canada|YXL|
|       Grand Rapids|   MI|    USA|GRR|
+-------------------+-----+-------+---+
only showing top 20 rows



In [ ]:
gf1.edges.show(10)

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
+-------+-----+--------+---+---+
only showing top 10 rows



### Determine the number of airports

In [ ]:
print("Airports number = ",gf1.vertices.count())

Airports number =  524


### Determine the number of trips 

In [ ]:
print("Trips number = ",gf1.edges.count())


Trips number =  1391578


### What is the longest delay?

In [ ]:
gf1.edges.groupBy().max("delay").show()

+----------+
|max(delay)|
+----------+
|      1642|
+----------+



### Find out the number of delayed flights vs. early flights (flights that departed before actual time)

In [ ]:
print("Number of delayed flights : ", gf1.filterEdges("delay > 0").edges.count())
print("Number of early flights : ", gf1.filterEdges("delay <= 0").edges.count())

Number of delayed flights :  591727
Number of early flights :  799851


### What flight destinations departing SFO are most likely to have significant delays? Select the top 10
#### Hint: you should get the average delay for each destination for trips that depart from SFO only

In [ ]:
gf1.edges.filter("src = 'SFO' and delay > 0").groupBy("dst").avg("delay").orderBy(col("avg(delay)").desc()).show(10)

+---+------------------+
|dst|        avg(delay)|
+---+------------------+
|OKC|59.073170731707314|
|JAC| 57.13333333333333|
|COS|53.976190476190474|
|OTH| 48.09090909090909|
|SAT|            47.625|
|MOD| 46.80952380952381|
|SUN|46.723404255319146|
|CIC| 46.72164948453608|
|ABQ|           44.8125|
|ASE|44.285714285714285|
+---+------------------+
only showing top 10 rows



### Find the Incoming connections to the airport sorted in Desc. order.

In [ ]:
gf1.inDegrees.sort(col("inDegree").desc()).show()

+---+--------+
| id|inDegree|
+---+--------+
|ATL|   90434|
|DFW|   66050|
|ORD|   61967|
|LAX|   53601|
|DEN|   50921|
|IAH|   42700|
|PHX|   39721|
|SFO|   38988|
|LAS|   32994|
|CLT|   28388|
|MCO|   27959|
|EWR|   27652|
|LGA|   25469|
|BOS|   25360|
|SLC|   25323|
|JFK|   23484|
|DTW|   23310|
|SEA|   23074|
|MSP|   22385|
|MIA|   21805|
+---+--------+
only showing top 20 rows



### Find the Outgoing connections from the airport sorted in Desc. order.

In [ ]:
gf1.outDegrees.sort(col("outDegree").desc()).show()

+---+---------+
| id|outDegree|
+---+---------+
|ATL|    91484|
|DFW|    68482|
|ORD|    64228|
|LAX|    54086|
|DEN|    53148|
|IAH|    43361|
|PHX|    40155|
|SFO|    39483|
|LAS|    33107|
|CLT|    28402|
|MCO|    28313|
|EWR|    27656|
|SLC|    25868|
|LGA|    25458|
|BOS|    25348|
|MSP|    24031|
|JFK|    23572|
|DTW|    23421|
|SEA|    23078|
|MIA|    21817|
+---+---------+
only showing top 20 rows



### Use motif finding to answer this question: which delays could we blame on SFO?
#### Hint: this practically means that SFO is a transit station

In [ ]:
motiv = gf1.find("(v1)-[e1]->(v2); (v2)-[e2]->(v3)").filter("(v2.id = 'SFO') and (e2.delay > 500 ) and (e2.tripid > e1.tripid)")
motiv.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  v1|                  e1|                  v2|                  e2|                  v3|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1211508, 593, 22...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[2092230, 636, 22...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[2092110, 740, 22...|[Miami, FL, USA, ...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[2190925, 1638, 1...|[Chicago, IL, USA...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[2030906, 516, 21...|[Philadelphia, PA...|
|[Albuquerque, NM,...|[1010600, -7, 7

### Determine Airport Ranking in Desc. order using PageRank algorithm

In [ ]:
ranks = gf1.pageRank(resetProbability=0.15, maxIter=5)

In [ ]:
ranks.vertices.orderBy(desc("pagerank")).show()

+--------------+-----+-------+---+------------------+
|          City|State|Country| id|          pagerank|
+--------------+-----+-------+---+------------------+
|       Atlanta|   GA|    USA|ATL|30.852689637281408|
|        Dallas|   TX|    USA|DFW|22.350908251857966|
|       Chicago|   IL|    USA|ORD|21.476110490648217|
|        Denver|   CO|    USA|DEN|15.864147080276448|
|   Los Angeles|   CA|    USA|LAX|14.200409160093672|
|       Houston|   TX|    USA|IAH|13.038206739080557|
| San Francisco|   CA|    USA|SFO| 11.26295269294737|
|       Phoenix|   AZ|    USA|PHX|10.614648412382879|
|Salt Lake City|   UT|    USA|SLC| 9.462289209001076|
|     Las Vegas|   NV|    USA|LAS|  8.57147332023463|
|       Seattle|   WA|    USA|SEA| 7.548959663023092|
|        Newark|   NJ|    USA|EWR| 7.180791090543531|
|       Orlando|   FL|    USA|MCO|7.1609265476583905|
|     Charlotte|   NC|    USA|CLT| 7.143654438927899|
|       Detroit|   MI|    USA|DTW| 6.890631890038126|
|   Minneapolis|   MN|    US

## Determine the most popular flights (single city hops)

In [ ]:
import pyspark.sql.functions as func

gf1.edges.groupBy("src","dst").agg(func.count("delay").alias("trips")).sort(desc("trips")).show()

+---+---+-----+
|src|dst|trips|
+---+---+-----+
|SFO|LAX| 3232|
|LAX|SFO| 3198|
|LAS|LAX| 3016|
|LAX|LAS| 2964|
|JFK|LAX| 2720|
|LAX|JFK| 2719|
|ATL|LGA| 2501|
|LGA|ATL| 2500|
|LAX|PHX| 2394|
|PHX|LAX| 2387|
|HNL|OGG| 2380|
|OGG|HNL| 2379|
|LAX|SAN| 2215|
|SAN|LAX| 2214|
|SJC|LAX| 2208|
|LAX|SJC| 2201|
|ATL|MCO| 2136|
|MCO|ATL| 2090|
|SFO|JFK| 2084|
|JFK|SFO| 2084|
+---+---+-----+
only showing top 20 rows



### Find and Save a Subragph that obtained from the following pattern:
#### The flight starts from an airport and return back to the same airport through 2 other airports.

In [ ]:
pattern_gf = gf1.find("(v1)-[e1]->(v2); (v2)-[e2]->(v3); (v3)-[e3]->(v1)")

In [ ]:
pattern_gf.printSchema()

root
 |-- v1: struct (nullable = false)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- Country: string (nullable = true)
 |    |-- id: string (nullable = true)
 |-- e1: struct (nullable = false)
 |    |-- tripid: integer (nullable = true)
 |    |-- delay: integer (nullable = true)
 |    |-- distance: integer (nullable = true)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |-- v2: struct (nullable = false)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- Country: string (nullable = true)
 |    |-- id: string (nullable = true)
 |-- e2: struct (nullable = false)
 |    |-- tripid: integer (nullable = true)
 |    |-- delay: integer (nullable = true)
 |    |-- distance: integer (nullable = true)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |-- v3: struct (nullable = false)
 |    |-- City: string (nullable = true)
 |    |-- State: stri